In [ ]:
# import mysql.connector

# # Connect to remote MySQL
# try:
#     conn = mysql.connector.connect(
#     host="192.168.194.204",
#     user="yash",
#     password="remote",
#     # database="swiggy_analysis",
#     database="swiggy_analysis_python",
#     port=3306  # default port, change if different
#     )
# except mysql.connector.Error as e:
#     print("Error connecting to the database:", e)
#     exit(1)


# cursor = conn.cursor()
# cursor.execute("SELECT VERSION();")
# version = cursor.fetchone()
# print("Database version:", version[0])

# # # Close



# # cursor.close()
# # conn.close()


Database version: 8.0.42


## Database Connection

In [ ]:
import mysql.connector
from mysql.connector import Error
import sys
import os

# Get the root directory (adjust if needed)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add it to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# Now try the import
from db.db_connection import SwiggyDBConnection


def connect_to_database(host, user, password, database):
    """
    Connects to the MySQL database and returns connection and cursor objects.

    Parameters:
        host (str): Database server address.
        user (str): Username.
        password (str): Password.
        database (str): Database name.

    Returns:
        conn: MySQL connection object
        cursor: MySQL cursor object

    Raises:
        Error: If connection fails
    """
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

        if conn.is_connected():
            print("Connected to MySQL database")
            cursor = conn.cursor()
            return conn, cursor

    except Error as e:
        print(f"Error connecting to database: {e}")
        return None, None
    



### Show Databases

In [3]:
def show_databases(cursor):
    try:
        cursor.execute("SHOW DATABASES;")
        databases = cursor.fetchall()
        print("Databases:")
        for db in databases:
            print(db[0])
    except Error as e:
        print(f"Error fetching databases: {e}")


conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    # Use cursor.execute() for queries
    
    show_databases(cursor)
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
Databases:
information_schema
mysql
performance_schema
smartcampus360
swiggy_analysis
sys


### Show Tables

In [4]:
def show_tables(cursor):
    try:
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print("Tables:")
        for table in tables:
            print(table[0])
    except Error as e:
        print(f"Error fetching tables: {e}")

conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    # Use cursor.execute() for queries
    
    try:
        show_tables(cursor)
        
    except Error as e:
        print(f"Error executing query: {e}")
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Tables:
booking
city
cuisines
delivery
fact_swiggy
locality
ratings


### Create Table and Trigger Functions

In [5]:
def create_table(cursor, create_table_query):
    try:
        cursor.execute(create_table_query)
        print("Table created successfully.")
    except Error as e:
        print(f"Error creating table: {e}")
        

def create_trigger(cursor, trigger_sql):
    try:
        cursor.execute(trigger_sql)
        print("Trigger created successfully.")
    except Error as e:
        print(f"Error creating trigger: {e}")

### Drop table function

In [23]:
def drop_table(cursor, table_name):
    """
    Drops the specified table if it exists.
    
    Parameters:
        cursor: MySQL cursor object
        table_name (str): Name of the table to drop
    """
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table '{table_name}' dropped successfully.")
    except Error as e:
        print(f"Error dropping table '{table_name}': {e}")

## Swiggy_source Table

### Create Swiggy_Source Table

In [6]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    create_swiggy_source_table_query = """
    CREATE TABLE swiggy_source (
        restaurant_name VARCHAR(255),
        city VARCHAR(100),
        locality VARCHAR(150),
        cuisines VARCHAR(100),
        average_cost_for_two INT,
        has_table_booking VARCHAR(3),
        has_online_delivery VARCHAR(3),
        rating_stars_out_of_5 DOUBLE,
        rating_in_text VARCHAR(20),
        price_range INT,
        votes INT
    );
    """
    create_table(cursor, create_swiggy_source_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    


Connected to MySQL database
Table created successfully.


### Load Swiggy_Source Table

In [12]:
def load_data_from_csv(cursor, csv_file_path):
    """
    Loads data from a CSV file into the swiggy_source table.

    Parameters:
        cursor: MySQL cursor object
        csv_file_path (str): Path to the CSV file
    """
    try:
        load_query = f"""
        LOAD DATA LOCAL INFILE '{csv_file_path}'
        INTO TABLE swiggy_source
        FIELDS TERMINATED BY ','
        ENCLOSED BY '"'
        LINES TERMINATED BY '\\n'
        IGNORE 1 ROWS;
        """
        cursor.execute(load_query)
        print("Data loaded successfully from CSV.")
    except Error as e:
        print(f"Error loading data from CSV: {e}")
        
        
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    # data_load_query = """
    # LOAD DATA LOCAL INFILE '/tmp/Swiggy_Analysis_Source_File.csv'
    # INTO TABLE swiggy_source
    # FIELDS TERMINATED BY ',' 
    # ENCLOSED BY '"'
    # LINES TERMINATED BY '\n'
    # IGNORE 1 ROWS
    # (restaurant_name, city, locality, cuisines, average_cost_for_two, has_table_booking, has_online_delivery, rating_stars_out_of_5, rating_in_text, price_range, votes);
    # """

    csv_file_path = '/var/lib/mysql-files/Swiggy_Analysis_Source_File.csv'
    load_data_from_csv(cursor, csv_file_path)

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Error loading data from CSV: 2068 (HY000): LOAD DATA LOCAL INFILE file request rejected due to restrictions on access.


### Drop Swiggy_Source Table

In [26]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "swiggy_source")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table 'swiggy_source' dropped successfully.


## Ratings Table

In [6]:
def create_table(cursor, create_table_query):
    try:
        cursor.execute(create_table_query)
        print("Table created successfully.")
    except Error as e:
        print(f"Error creating table: {e}")
        

def create_trigger(cursor, trigger_sql):
    try:
        cursor.execute(trigger_sql)
        print("Trigger created successfully.")
    except Error as e:
        print(f"Error creating trigger: {e}")

### Create Ratings Table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    # Create 'ratings' table if it does not exist

    create_rating_table_query = '''
        CREATE TABLE IF NOT EXISTS ratings (
            id INT PRIMARY KEY AUTO_INCREMENT, 
            star DOUBLE NOT NULL, 
            text VARCHAR(20) NOT NULL
        );
    '''
    create_table(cursor, create_rating_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table created successfully.


### Create set_rating_text Trigger

In [8]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    ## DROP trigger if it already exists to avoid duplicate error
    cursor.execute("DROP TRIGGER IF EXISTS set_rating_text")

    ## Define trigger using multiline string
    trigger_sql = """
    CREATE TRIGGER set_rating_text
    BEFORE INSERT ON ratings
    FOR EACH ROW
    BEGIN
        IF NEW.star > 4.4 THEN
            SET NEW.text = 'Excellent';
        ELSEIF NEW.star > 3.9 THEN
            SET NEW.text = 'Very Good';
        ELSEIF NEW.star > 3.4 THEN
            SET NEW.text = 'Good';
        ELSEIF NEW.star > 1.9 THEN
            SET NEW.text = 'Average';
        ELSE
            SET NEW.text = 'Low';
        END IF;
    END
    """

    cursor.execute(trigger_sql)
        
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database


### Initialize Ratings Table

In [9]:
def initialize_ratings_table(cursor):
    """
    Initializes the ratings table with values from 0.1 to 5.0 in steps of 0.1.
    """
    try:
        cursor.execute("DELETE FROM ratings")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        for i in range(50, 0, -1):  # 0.1 to 5.0 in steps of 0.1
            star = round(i * 0.1, 1)
            cursor.execute("INSERT INTO ratings (star) VALUES (%s)", (star,))
        conn.commit()
        print("Ratings table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing ratings table: {e}")


conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_ratings_table(cursor)
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Ratings table initialized successfully.


### Drop Ratings Table

In [65]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "ratings")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Error connecting to database: 2003 (HY000): Can't connect to MySQL server on '192.168.194.204:3306' (113)
Failed to connect to database


### Select Data from Tables

In [10]:
def select_data_from_table(cursor, table_name):
    """
    Selects all data from the specified table.
    
    Parameters:
        cursor: MySQL cursor object
        table_name (str): Name of the table to select data from
    """
    try:
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()
        for row in rows:
            print(row)
    except Error as e:
        print(f"Error selecting data from {table_name}: {e}")


def select_data_from_table_with_limit(cursor, table_name, limit):
    """
    Selects data from a specified table with an optional limit.
    
    Parameters:
        cursor: MySQL cursor object
        table_name (str): Name of the table to select data from
        limit (int): Number of rows to return
    
    Returns:
        list: List of tuples containing the selected rows
    """
    try:
        query = f"SELECT * FROM {table_name} LIMIT {limit};"
        cursor.execute(query)
        return cursor.fetchall()
    except Error as e:
        print(f"Error selecting data from table {table_name}: {e}")
        return []

### Select Data from Ratings

In [11]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    ratings = select_data_from_table_with_limit(cursor, "ratings", limit=10)
    for row in ratings:
        print(row)
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
(1, 5.0, 'Excellent')
(2, 4.9, 'Excellent')
(3, 4.8, 'Excellent')
(4, 4.7, 'Excellent')
(5, 4.6, 'Excellent')
(6, 4.5, 'Excellent')
(7, 4.4, 'Very Good')
(8, 4.3, 'Very Good')
(9, 4.2, 'Very Good')
(10, 4.1, 'Very Good')


### Drop Trigger function

In [ ]:
def drop_trigger(cursor, trigger_name):
    """
    Drops the specified trigger if it exists.
    
    Parameters:
        cursor: MySQL cursor object
        trigger_name (str): Name of the trigger to drop
    """
    try:
        cursor.execute(f"DROP TRIGGER IF EXISTS {trigger_name};")
        print(f"Trigger '{trigger_name}' dropped successfully.")
    except Error as e:
        print(f"Error dropping trigger '{trigger_name}': {e}")

### Drop Trigger and Table for Ratings

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_trigger(cursor, "set_rating_text")
    drop_table(cursor, "ratings")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

## City Table

### Create City Table

In [12]:

conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    
    # Create 'city' table if it does not exist

    create_city_table_query = '''
    CREATE TABLE IF NOT EXISTS city (
        id INT PRIMARY KEY AUTO_INCREMENT, 
        name VARCHAR(100) NOT NULL
    );
    '''
    create_table(cursor, create_city_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table created successfully.


### Initialize City Table

In [18]:
def initialize_city_table(cursor):
    """
    Initializes the city table with predefined city names.
    """
    try:
        cursor.execute("DELETE FROM city")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        cities = [
            "Bangalore",
            "Chandigarh",
            "Chennai",
            "Goa",
            "Guwahati",
            "Hyderabad",
            "Jaipur",
            "Kolkata",
            "Lucknow",
            "Mumbai",
            "Patna",
            "Pune",
            "Surat"
        ]

        for city in cities:
            cursor.execute("INSERT INTO city (name) VALUES (%s)", (city,))
        conn.commit()
        print("City table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing city table: {e}")


conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_city_table(cursor)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
City table initialized successfully.


In [19]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    city = select_data_from_table(cursor, "city")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
(1, 'Bangalore')
(2, 'Chandigarh')
(3, 'Chennai')
(4, 'Goa')
(5, 'Guwahati')
(6, 'Hyderabad')
(7, 'Jaipur')
(8, 'Kolkata')
(9, 'Lucknow')
(10, 'Mumbai')
(11, 'Patna')
(12, 'Pune')
(13, 'Surat')


### Drop City Table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "city")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

## Cuisines Table

### Create Cuisines Table

In [20]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    
    # Create 'cuisines' table if it does not exist

    create_cuisines_table_query = '''
    CREATE TABLE IF NOT EXISTS cuisines (
        id INT PRIMARY KEY AUTO_INCREMENT, 
        name VARCHAR(100) NOT NULL
    );
    '''
    create_table(cursor, create_cuisines_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table created successfully.


### Initialize the cuisines table

In [21]:
def initialize_cuisines_table(cursor):
    """
    Initializes the cuisines table with predefined cuisine names.
    """
    try:
        cursor.execute("DELETE FROM cuisines")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        cuisines = [
            "American",
            "Asian",
            "Bakery",
            "Bengali",
            "Biryani",
            "Café",
            "Continental",
            "Desserts",
            "European",
            "Fast Food",
            "Healthy Food",
            "Hyderabadi",
            "Italian",
            "Japanese",
            "Mediterranean",
            "Mexican",
            "Modern Indian",
            "North Indian",
            "Rajasthani",
            "Seafood",
            "South Indian",
            "Street Food",
            "Goan",
            "Chinese",
            "Mughlai",
            "Charcoal Grill",
            "Lebanese",
            "Gujarati"
        ]

        for cuisine in cuisines:
            cursor.execute("INSERT INTO cuisines (name) VALUES (%s)", (cuisine,))
        conn.commit()
        print("Cuisines table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing cuisines table: {e}")


conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_cuisines_table(cursor)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
Cuisines table initialized successfully.


### Drop cuisines table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "cuisines")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

### Select data from cuisines table

In [22]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    select_data_from_table(cursor, "cuisines")
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
(1, 'American')
(2, 'Asian')
(3, 'Bakery')
(4, 'Bengali')
(5, 'Biryani')
(6, 'Café')
(7, 'Continental')
(8, 'Desserts')
(9, 'European')
(10, 'Fast Food')
(11, 'Healthy Food')
(12, 'Hyderabadi')
(13, 'Italian')
(14, 'Japanese')
(15, 'Mediterranean')
(16, 'Mexican')
(17, 'Modern Indian')
(18, 'North Indian')
(19, 'Rajasthani')
(20, 'Seafood')
(21, 'South Indian')
(22, 'Street Food')
(23, 'Goan')
(24, 'Chinese')
(25, 'Mughlai')
(26, 'Charcoal Grill')
(27, 'Lebanese')
(28, 'Gujarati')


## Delivery

### Create Delivery Table

In [23]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    """
    Creates the delivery table if it does not exist.
    """
    create_delivery_table_query = '''
    CREATE TABLE IF NOT EXISTS delivery (
        id INT PRIMARY KEY AUTO_INCREMENT, 
        availability VARCHAR(3) NOT NULL
    );
    '''
    create_table(cursor, create_delivery_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table created successfully.


### Initialize delivery table

In [24]:
def initialize_delivery_table(cursor):
    """
    Initializes the delivery table with predefined delivery availability.
    """
    try:
        cursor.execute("DELETE FROM delivery")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        deliveries = [
            ("Yes",),
            ("No",)
        ]

        cursor.executemany("INSERT INTO delivery (availability) VALUES (%s)", deliveries)
        conn.commit()
        print("Delivery table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing delivery table: {e}")
        

conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_delivery_table(cursor)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Delivery table initialized successfully.


### Drop the delivery table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "delivery")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

### Select the delivery table

In [25]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    select_data_from_table(cursor, "delivery")
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
(1, 'Yes')
(2, 'No')


## Locality Table

### Create Locality Table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    create_locality_table_query = '''
    CREATE TABLE IF NOT EXISTS locality (
        id INT PRIMARY KEY AUTO_INCREMENT, 
        name VARCHAR(150) NOT NULL
    );
    '''
    create_table(cursor, create_locality_table_query)
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Table created successfully.


### Initialization of Locality Table

In [ ]:
def initialize_locality_table(cursor):
    """
    Initializes the locality table with predefined locality names.
    """
    try:
        cursor.execute("DELETE FROM locality")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        locality_initialization_query = ''' 
        INSERT INTO locality (name)
        SELECT DISTINCT locality
        FROM swiggy_source
        WHERE locality IS NOT NULL;
        '''

        cursor.execute(locality_initialization_query)
        conn.commit()
        print("Locality table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing locality table: {e}")
        

conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_locality_table
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Locality table initialized successfully.


### Drop the Locality Table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "locality")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

### Select data from Locality Table

In [29]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    locality = select_data_from_table_with_limit(cursor, "locality", limit=10)
    for row in locality:
        print(row)
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
(1, 'BluPetal Hotel, Koramangala')
(2, 'Indiranagar')
(3, 'JP Nagar')
(4, 'Koramangala 5th Block')
(5, 'Koramangala 6th Block')
(6, 'Koramangala 7th Block')
(7, 'Marathahalli')
(8, 'New BEL Road')
(9, 'Residency Road')
(10, 'Sarjapur Road')


## Restaurant Table

### Create Restaurant Table

In [31]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    create_restaurants_table_query = '''
    CREATE TABLE IF NOT EXISTS restaurants (
        id INT PRIMARY KEY AUTO_INCREMENT, 
        name VARCHAR(255) NOT NULL
    );
    '''
    create_table(cursor, create_restaurants_table_query)
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Table created successfully.


### Initialization of Restaurants Table

In [34]:
def initialize_restaurants_table(cursor):
    """
    Initializes the restaurants table with restaurant names from swiggy_source.
    """
    try:
        cursor.execute("DELETE FROM restaurants")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        restaurants_initialization_query = ''' 
        INSERT INTO restaurants (name)
        SELECT DISTINCT restaurant_name
        FROM swiggy_source
        WHERE restaurant_name IS NOT NULL;
        '''

        cursor.execute(restaurants_initialization_query)
        conn.commit()
        print("Restaurants table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing restaurants table: {e}")

        

conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_restaurants_table(cursor)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Restaurants table initialized successfully.


### Drop Restaurants Table

In [ ]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    drop_table(cursor, "restaurants")
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

### Select data from Restaurants Table

In [35]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    restaurants = select_data_from_table_with_limit(cursor, "restaurants", limit=10)
    for row in restaurants:
        print(row)
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
(1, 'Sultans of Spice')
(2, 'The Fatty Bao - Asian Gastro Bar')
(3, 'Toit')
(4, 'Three Dots & A Dash')
(5, 'Bombay Brasserie')
(6, "Glen's Bakehouse")
(7, 'Onesta')
(8, 'ECHOES Koramangala')
(9, 'Truffles')
(10, 'The Black Pearl')


## Booking Table

### Create a Booking Table

In [32]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    create_booking_table_query = '''
    CREATE TABLE IF NOT EXISTS booking (
        id INT PRIMARY KEY AUTO_INCREMENT,
        availability VARCHAR(3) NOT NULL
    );
    '''
    create_table(cursor, create_booking_table_query)
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Table created successfully.


### Initialization of Booking Table

In [36]:
def initialize_booking_table(cursor):
    """
    Initializes the booking table with predefined booking availability.
    """
    try:
        cursor.execute("DELETE FROM booking")  # Clear existing data before inserting new data
        conn.commit()  # Commit the delete operation

        bookings = [
            ("Yes",),
            ("No",)
        ]

        cursor.executemany("INSERT INTO booking (availability) VALUES (%s)", bookings)
        conn.commit()
        print("Booking table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing booking table: {e}")
        
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_booking_table(cursor)
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")


Connected to MySQL database
Booking table initialized successfully.


## Fact_swiggy Table

In [37]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    """
    Creates the fact_swiggy table if it does not exist.
    """
    
    create_fact_swiggy_table_query = '''
    CREATE TABLE IF NOT EXISTS fact_swiggy (
        fact_id INT PRIMARY KEY AUTO_INCREMENT, 
        city_id INT NOT NULL, 
        locality_id INT NOT NULL,
        rest_id INT NOT NULL,
        cuisine_id INT NOT NULL, 
        rating_id INT NOT NULL,
        delivery_id INT NOT NULL, 
        booking_id INT NOT NULL,
        avg_cost_for_two INT NOT NULL,
        votes INT NOT NULL,
        price_range INT NOT NULL,
        FOREIGN KEY (city_id) REFERENCES city(id),
        FOREIGN KEY (locality_id) REFERENCES locality(id),
        FOREIGN KEY (rest_id) REFERENCES restaurants(id),
        FOREIGN KEY (cuisine_id) REFERENCES cuisines(id),
        FOREIGN KEY (rating_id) REFERENCES ratings(id),
        FOREIGN KEY (delivery_id) REFERENCES delivery(id),
        FOREIGN KEY (booking_id) REFERENCES booking(id)
    );
    '''
    create_table(cursor, create_fact_swiggy_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table created successfully.


In [21]:
# Remote
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    """
    Creates the fact_swiggy table if it does not exist.
    """
    
    create_fact_swiggy_table_query = '''
    CREATE TABLE IF NOT EXISTS fact_swiggy (
        fact_id INT PRIMARY KEY AUTO_INCREMENT, 
        city_id INT NOT NULL, 
        locality_id INT NOT NULL,
        rest_id INT NOT NULL,
        cuisine_id INT NOT NULL, 
        rating_id INT NOT NULL,
        delivery_id INT NOT NULL, 
        book_id INT NOT NULL,
        avg_cost_for_2 INT NOT NULL,
        votes INT NOT NULL,
        price_range INT NOT NULL,
        FOREIGN KEY (city_id) REFERENCES city_dim(city_id),
        FOREIGN KEY (locality_id) REFERENCES locality_dim(locality_id),
        FOREIGN KEY (rest_id) REFERENCES restaurant(rest_id),
        FOREIGN KEY (cuisine_id) REFERENCES cuisines(cuisine_id),
        FOREIGN KEY (rating_id) REFERENCES ratings(rating_id),
        FOREIGN KEY (delivery_id) REFERENCES delivery(delivery_id),
        FOREIGN KEY (book_id) REFERENCES table_booking(book_id)
    );
    '''
    create_table(cursor, create_fact_swiggy_table_query)
    

    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")
    

Connected to MySQL database
Table created successfully.


### Initialize the fact_swiggy table

In [ ]:
def initialize_fact_table_from_swiggy_source(cursor):
    """
    Initializes the fact table with data from the source table.
    """
    try:
        
        fact_table_inilization_query = '''
        INSERT INTO fact_swiggy (
            city_id, locality_id, rest_id, cuisine_id, rating_id,
            delivery_id, booking_id, avg_cost_for_two, votes, price_range
        )
        SELECT
            c.id, 
            l.locality_id, 
            r.rest_id, 
            cu.cuisine_id, 
            rt.rating_id,
            d.delivery_id, 
            b.booking_id,
            s.average_cost_for_two,
            s.votes,
            s.price_range
        FROM swiggy_source s
        JOIN city c ON s.city = c.name
        JOIN locality l ON s.locality = l.name
        JOIN restaurant r ON s.restaurant_name = r.name
        JOIN cuisine cu ON s.cuisines = cu.name
        JOIN rating rt ON s.rating_stars_out_of_5 = rt.stars AND s.rating_in_text = rt.text
        JOIN delivery d ON s.has_online_delivery = d.availability
        JOIN booking b ON s.has_table_booking = b.availability;
        '''
        
        # Insert data into the fact table
        cursor.execute(fact_table_inilization_query)
        conn.commit()
        print("Fact table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing fact table: {e}")


conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_fact_table_from_swiggy_source(cursor)    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Error initializing fact table: 1146 (42S02): Table 'swiggy_python.city' doesn't exist


In [25]:
def initialize_fact_table_from_swiggy_source(cursor):
    """
    Initializes the fact table with data from the source table.
    """
    try:
        
        fact_table_inilization_query = '''
        INSERT INTO fact_swiggy (
            city_id, locality_id, rest_id, cuisine_id, rating_id,
            delivery_id, book_id, avg_cost_for_2, votes, price_range
        )
        SELECT
            c.city_id, 
            l.locality_id, 
            r.rest_id, 
            cu.cuisine_id, 
            rt.rating_id,
            d.delivery_id, 
            b.book_id,
            s.avg_cost_for_two,
            s.votes,
            s.price_range
        FROM swiggy_source s
        JOIN city_dim c ON s.city = c.city_name
        JOIN locality_dim l ON s.locality = l.locality_name
        JOIN restaurant r ON s.restaurant_name = r.rest_name
        JOIN cuisines cu ON s.cuisines = cu.cuisine_name
        JOIN ratings rt ON s.rating_stars_out_of_5 = rt.star AND s.rating_in_text = rt.rating_text
        JOIN delivery d ON s.has_online_delivery = d.delivery_avail
        JOIN table_booking b ON s.has_table_booking = b.book_avail;
        '''
        
        # Insert data into the fact table
        cursor.execute(fact_table_inilization_query)
        conn.commit()
        print("Fact table initialized successfully.")
        
    except Error as e:
        print(f"Error initializing fact table: {e}")


conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    initialize_fact_table_from_swiggy_source(cursor)    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database
Fact table initialized successfully.


In [20]:
conn, cursor = connect_to_database(host, user, password, database)

if conn and cursor:
    
    ### Drop the fact_swiggy table
    ## Code to drop the table if needed
    ## Uncomment the following lines if you want to drop the table

    cursor.execute("DROP TABLE IF EXISTS fact_swiggy")
    
    
    # Don't forget to close after done:
    cursor.close()
    conn.close()
else:
    print("Failed to connect to database")

Connected to MySQL database


In [ ]:
cursor.execute("")
conn.commit()

In [ ]:
# Close
cursor.close()
conn.close()